In [23]:
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin
from ceva import Ceva

from scipy.spatial.transform import Rotation as R
import numpy as np

from sympy import symbols, diff, sqrt

N = 5
dt = 1

# t = np.arange(0, 10, dt/10)
# traj = Ceva(N, dt)
# traj.genRandomTrajectory(100)

# theta = traj.getPose(t)[:, [1,2,3]]
# thetad = traj.getTransVelWorld(t)[:, [1,2,3]]
# thetadd = traj.getTransAccelWorld(t)[:, [1,2,3]]

def skew(v):
    v = v.reshape((3,))
    vsk = np.array([[ 0,    -v[2],  v[1]],
                    [ v[2],  0,    -v[0]],
                    [-v[1],  v[0],  0   ]]).reshape((3,3))
    return vsk

def g(V):
    Vp2 = V*V
    return (1 - cos(V))/Vp2

def gV(V):
    Vp2 = V*V
    Vp3 = Vp2*V
    return sin(V)/Vp2 + 2*(cos(V) - 1)/Vp3

def gVV(V):
    Vp2 = V*V
    Vp3 = Vp2*V
    Vp4 = Vp3*V
    return cos(V)/Vp2 - 4*sin(V)/Vp3 + 6*(1 - cos(V))/Vp4

def h(V):
    Vp3 = V*V*V
    return (V - sin(V))/Vp3

def hV(V):
    Vp3 = V*V*V
    Vp4 = Vp3*V
    return -2/Vp3 - cos(V)/Vp3 + 3*sin(V)/Vp4

def hVV(V):
    Vp2 = V*V
    Vp3 = Vp2*V
    Vp4 = Vp3*V
    Vp5 = Vp5*V
    return sin(V)/Vp3 + 6*(cos(V) + 1)/Vp4 - 12*sin(V)/Vp5

def gp(V):
    Vp2 = V*V
    return 1/Vp2 - (1+cos(V))/(2*V*sin(V))

def gpV(V):
    Vp2 = V*V
    Vp3 = Vp2*V
    return -2/Vp3 + (1+cos(V))*(V+sin(V))/(2*Vp2*sin(V)*sin(V))

def gpVV(V):
    Vp2 = V*V
    Vp3 = Vp2*V
    Vp4 = Vp3*V
    return 6/Vp4 + sin(V)/(Vp3*(cos(V) - 1)) + (V*cos(V) + 2*sin(V) + V)/(2*Vp2*sin(V)*(cos(V)-1))


def f(u, v):
    return skew(u)@skew(u)@v

def fv(u, v):
    return skew(u)@skew(u)
    
def fu(u, v):
    return -skew(u)@skew(v) - skew(skew(u)@v)

def fuu(u, v, w):
    return skew(skew(v)@w) - skew(w)@skew(v)

def fuv(u, v, w):
    return skew(u)@skew(w) + skew(w)@skew(u)


def SO3Exp(v):
    V = np.linalg.norm(v)
    u = v/V
    R = np.identity(3) + sin(V)*skew(u) + g(V)*skew(v)@skew(v)
    return R

def SO3Jr(v):
    V   = np.linalg.norm(v)
    # Vp2 = V*V
    # Vp3 = Vp2*V
    # u   = v/V
    vsk = skew(v)
    return np.identity(3) - g(V)*vsk + h(V)*vsk@vsk

def SO3JrInv(v):
    V   = np.linalg.norm(v)
    # Vp2 = V*V
    # u   = v/V
    vsk = skew(v)
    return np.identity(3) + 0.5*vsk + gp(V)*vsk@vsk

def SO3Jl(v):
    return SO3Jr(-v)

def SO3JlInv(v):
    return SO3JrInv(-v)

def Hu(u, v):
    un = np.linalg.norm(u)
    ub = u/un
    return skew(v)*g(un) + skew(v)@u@ub.T*gV(un) + fu(u, v)*h(un) + f(u, v)@ub.T*hV(un)

def Hpu(u, v):
    un = np.linalg.norm(u)
    ub = u/un
    return -0.5*skew(v) + fu(u, v)*gp(un) + f(u, v)@ub.T*gpV(un)


X  = np.array([43, 57, 91.0], dtype=float).reshape((3, 1))
Xd = np.array([11, 2, 19]).reshape((3, 1))
O  = SO3Jr(X)@Xd

# print(SO3Jr(X))
# print(SO3JrInv(X))
# print(SO3Jr(X) @ SO3JrInv(X))

# print(Hu(X, Xd))
# print(O)
# print(Hpu(X, O))
# print(Hpu(X, SO3Jr(X)@Xd))
print(Hpu(X, SO3Jr(X)@Xd) + SO3JrInv(X)@Hu(X, Xd))
print(Hpu(X, O) + SO3JrInv(X)@Hu(X, SO3JrInv(X)@O))

# print(SO3Jr(X)@skew(X)@SO3JrInv(X) - skew(SO3Jr(X)@X))
# print(Hpu(X, Xd)@SO3Jr(X) + SO3JrInv(X)@Hu(X, Xd))

# print(SO3Jr(X))
# print(SO3Exp(-X)@SO3Jl(X))
# print(SO3JrInv(X) - SO3JlInv(X)@SO3Exp(X))

# The_x, They, Thez = symbols('The_x The_y The_z')
# Thedx, Thedy, Thedz = symbols('Thex They Thez')

# The = np.array([Thex, They, Thez]).reshape((3, 1))
# Thed = np.array([Thedx, Thedy, Thedz]).reshape((3, 1))

[[ 1.82076576e-14  1.95399252e-14  3.73034936e-14]
 [-2.48689958e-14 -3.37507799e-14 -5.77315973e-14]
 [ 4.44089210e-15  7.10542736e-15  1.15463195e-14]]
[[ 1.15463195e-14  1.42108547e-14  2.64233080e-14]
 [-1.33226763e-14 -2.17603713e-14 -3.46389584e-14]
 [ 0.00000000e+00  2.66453526e-15  2.66453526e-15]]


In [ ]:

# fig = plt.figure(figsize=(16, 4))
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(theta[:, 0], theta[:, 1], theta[:, 2], label='GNDTRUTH ', color='r')